In [2]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
from pympler import asizeof

In [12]:
# scv=pd.read_csv('ratings.csv', index_col=False)  
# keys=scv["timestamp"]
# sort_keys=keys.sort_values(axis=0).drop_duplicates()
# sort_keys.reset_index(drop= True , inplace= True)

scv=pd.read_csv("cinema.csv", index_col=False)
sort_keys=scv["timestamp"]
# import base36
data=[[key,ind] for ind,key in enumerate(sort_keys)]

print(len(data), data[1])

25976523 [789652009, 1]


In [ ]:
class GreedySpline:
    def __init__(self, error=10):
        self.error=error
        self.greedyspline=[]

    def start(self,new_point):
        self.tg_err_top=float("inf")
        self.tg_err_bot=float("-inf")
        self.greedyspline.append(new_point)
        self.greedyspline.append(new_point)
        self.last_point=new_point
    
    def tg(self, a,b,err=0):
        return (a[1]+self.error*err-b[1])/(a[0]-b[0])
        
    def new_coridor (self,a,b):
        self.tg_err_top=min(self.tg_err_top, self.tg(a,b,1))
        self.tg_err_bot=max(self.tg_err_bot, self.tg(a,b,-1))
        
    def new_spline(self, new_point): 
        if not self.greedyspline:
            self.start(new_point)
            return
        tg_new_point=self.tg(new_point,self.greedyspline[-2])
        if tg_new_point<self.tg_err_top and tg_new_point>self.tg_err_bot:
            self.new_coridor(new_point, self.greedyspline[-2])
            self.greedyspline[-1]=new_point
        else:
            self.greedyspline.append(self.last_point)
            self.tg_err_top=self.tg(new_point, self.greedyspline[-1],1)
            self.tg_err_bot=self.tg(new_point, self.greedyspline[-1],-1)
        self.last_point=new_point 
        
        
        
    def load_data(self,data):
        for i in data:
            self.new_spline(i)
    
    def plot(self):
        dataX=[]
        dataY=[]
        for i,j in  self.greedyspline:
            dataX.append(i)
            dataY.append(j)
        plt.plot(dataX,dataY, marker='o', markersize=6)
    
        
        
    
class RadixSpline:
    def __init__(self,GreedySpline,bitcount):
        self.bitcount=bitcount
        self.greedyspline=GreedySpline.greedyspline
        self.error=GreedySpline.error
        self.radixspline=[None]*2**self.bitcount
        self.bit_len=self.bit_lenght(self.greedyspline[-1][0])
        # print("bit_len",self.bit_len)
        self.len_greedy=len(self.greedyspline)
        self.not_prefics,self.mod=self.not_prefics_count()
        self.move_right=self.bit_len-self.bitcount-self.not_prefics
        # print("not_prefics",self.not_prefics)
        # print("mod", self.mod)
        # print("move_right",self.move_right)
        self.get_RadixSpline()
        
    def bit_lenght(self,n):
        return(len(bin(n).lstrip('-0b')))
    def not_prefics_count(self):
        n=self.greedyspline[0][0]
        count,pref=0,0
        
        if self.bit_lenght(n)==self.bit_len:
            count=self.bit_len-self.bit_lenght(((1<<self.bit_len)-1)^n)
        for i in range(count):
            pref=(pref<<1)+1
        pref<<=(self.bit_len-count)
        return count,pref
        
        
    
    ##Способ построение радикс сплайна, нужно заменить
    def first_bits(self,n):
        if self.not_prefics:
           n%=self.mod
        #    print(n)
        return n>>self.move_right
    
    def get_RadixSpline(self):
        
        for ind in range(len(self.greedyspline)):
            temp=self.first_bits(self.greedyspline[ind][0])
            # print(temp)
            if not self.radixspline[temp]:
                self.radixspline[temp]=ind
        if  self.radixspline[2**self.bitcount-1]==None:
             self.radixspline[2**self.bitcount-1]=len(self.greedyspline)-1
        for i in range (2**self.bitcount-1,-1,-1):
            if self.radixspline[i]==None:                
                self.radixspline[i]=self.radixspline[i+1]
        return self.radixspline
    
    def bin_search_spline(self,left,right,key):
        while left<=right:
            mid=(left+right)//2
            if key>= self.greedyspline[mid][0] :
                if key<=self.greedyspline[mid+1][0] :
                   return [mid,mid+1]
                else: 
                   left=mid+1
            else:
                right=mid-1


    
        
    def give_answer(self,key):
        first_bit=self.first_bits(key)
        left=self.radixspline[first_bit]-1
        if first_bit==len(self.radixspline)-1:
            right=len(self.greedyspline)-1
        else:
            right=self.radixspline[first_bit+1]
        # left=0
        # right=len(self.greedyspline)-1
        bin_ans=self.bin_search_spline(left,right,key)
        kl,pl=self.greedyspline[bin_ans[0]]
        kr,pr=self.greedyspline[bin_ans[1]]

        return pl+(key-kl)*(pr-pl)//(kr-kl)
    
    def plot(self):
        dataX=[]
        dataY=[]
        for i in  self.radixspline:
            dataX.append(self.greedyspline[i][0])
            dataY.append(self.greedyspline[i][1])
        plt.plot(dataX,dataY, marker='o', markersize=15)

In [14]:
def plot_data(data,error=0):
    dataX=[]
    errT=[]
    errB=[]
    dataY=[]
    for i,j in  data:
        dataX.append(i)
        dataY.append(j)
        if error:
            errB.append(j-error)
            errT.append(j+error)
    plt.plot(dataX,dataY,marker='o', markersize=2)
    print(data.__sizeof__())
    if error:
        plt.plot(dataX,errB,marker='o', markersize=2)
        plt.plot(dataX,errT,marker='o', markersize=2)      

def get_spline(data, ERR,bitcount ,plot=False): 
    G=GreedySpline(error=ERR)
    G.load_data(data)
    
    # print(len(G.greedyspline)/len(data)) 
    R=RadixSpline(G, bitcount)
    if plot:
        R.plot()
        G.plot()
        plot_data(data)
        plt.show()
    return (G,R)

In [15]:
# scv=pd.read_csv('amazon_com.csv', dtype={"ASIN":bytes}, index_col=False)  
# keys=scv["ASIN"]
# sort_keys=keys.sort_values(axis=0).drop_duplicates()
# sort_keys.reset_index(drop= True , inplace= True)
# # sort_keys.to_csv("amazon.csv")
# sort_keys[1]
# import base36
# data=[[base36.loads(key),ind] for ind,key in enumerate(sort_keys)]
# data=data

In [16]:
def bin_search(data, key, left, right):
    while left <= right:
        mid = (left + right) // 2  # Находим середину массив
        if data[mid][0] == key:  # Если целевое значение найдено
            return mid
        elif data[mid][0] < key:  # Если целевое значение справа
            left = mid + 1
        else:  # Если целевое значение слева
            right = mid - 1
    return -1  

    
def get_key(data, spline, key ):
    radix_answer=spline[1].give_answer(key[0])
    left=radix_answer-spline[0].error
    right=radix_answer+spline[0].error
    res=bin_search(data, key[0], max(left,data[0][1]),min(right, data[-1][1]))
    # print(res)
    if res!=key[1]: print ("FAIL", res, key, math.floor(left),math.ceil(right))
    return res
  
def check_bin_search(data, key):
    res=bin_search(data, key[0],data[0][1], data[-1][1])
    if res!=key[1]: print ("FAIL binsearch")
    return res

## Тестирование сплайна

создание одного

In [ ]:
bitcount=5
error=2**14
A=get_spline(data,16,20)

Прогон, поиск всех ключей, для проверки приблизительного времени 

In [49]:
for i in tqdm(data):
    get_key(data,A,i)

  0%|          | 0/25976523 [00:00<?, ?it/s]

Сравнение с приблизительным временем бинпоиска

In [50]:
for i in tqdm(data):
    check_bin_search(data,i)

  0%|          | 0/25976523 [00:00<?, ?it/s]

Перебор параметров, для проверки времени

In [47]:
for i in range(8,18):
    print("i=", i)
    start1=time.time()
    A=get_spline(data,2**i,5)
    end1 =time.time()
    start2=time.time()
    for i in data:
        get_key(data,A,i)
    end2=time.time()
    print("gen",(end1-start1)//0.001,"|search",(end2-start2)//0.001,"|greedy_size",A[1].len_greedy, "size(bytes)= ", asizeof.asizeof(A) )
    
    print("_"*14)
    

i= 10
gen 25.739999294281006 |search 70.93499827384949 |greedy_size 101
______________


"Эталонное" время бинпоиска

In [43]:
start2=time.time()
for i in data:    
    check_bin_search(data,i)
end2=time.time()
print(-start2+end2)

75.37800550460815


Создание Фенс поинтов

In [4]:
scv=pd.read_csv('ratings.csv', index_col=False)  
keys=scv["timestamp"]
sort_keys=keys.sort_values(axis=0).drop_duplicates()
sort_keys.reset_index(drop= True , inplace= True)

In [60]:
case_size=4096
len_points=os.path.getsize("ratings.csv")//case_size
average_size=os.path.getsize("ratings.csv")//len(scv)
points=[]
print(average_size)
for i in range (0,len(sort_keys),case_size//average_size ):
    points.append(sort_keys[i])

27


Отчет по используемой памяти

In [64]:
print(os.path.getsize("rating.csv"),"       size ratings")
print(os.path.getsize("cinema.csv"),"       size cinema (обрезанный ratings)")

print( asizeof.asizeof(get_spline(data,16,20)), "  =  asize 16,20 ")
print( asizeof.asizeof(get_spline(data,2**14,8)),"  =  asize 2**14,8 ")
print(asizeof.asizeof(points), "     =asize points")

877076222        size ratings
531418210        size cinema (обрезанный ratings)
65916048   =  asize 16,20 
20752   =  asize 2**14,8 
6948552      =asize points


In [22]:
scv=pd.read_csv('rating.csv', index_col=False)  
keys=scv["timestamp"]
sort_keys=keys.sort_values(axis=0).drop_duplicates()
sort_keys.reset_index(drop= True , inplace= True)

In [23]:
sort_keys.to_csv("cinema.csv")

In [12]:
pd.read_csv('cinema.csv', index_col=False)

,Unnamed: 0,timestamp
0,0,789652004
1,1,789652009
2,2,822873600
3,3,823185196
4,4,823185197
...,...,...
2999995,2999995,1001110297
2999996,2999996,1001110308
2999997,2999997,1001110332
2999998,2999998,1001110335


In [21]:
scv=pd.read_csv('ratings.csv', index_col=False)  
scv[:76*10**4].to_csv("rating.csv")
os.path.getsize("rating.csv")

24760980

In [24]:
os.path.getsize("cinema.csv")

11786810